# Recomendador de libros

In [1]:
#Importamos las librerias (luego voy agregando aqui)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import datetime
import re

Primero importamos y limpiamos el archivo de goodreads

In [2]:
data = pd.read_csv("goodreads.csv")

In [3]:
data.head()

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
0,127305713,"Heartless Hunter (The Crimson Moth, #1)",Kristen Ciccarelli,"Ciccarelli, Kristen",NaN,"=""1250866901""","=""9781250866905""",0,4.25,Wednesday Books,...,NaN,2024/05/08,"to-read, fantasy","to-read (#116), fantasy (#71)",to-read,NaN,NaN,NaN,0,0
1,115821328,Una familia casi perfecta,Jane Shemilt,"Shemilt, Jane",NaN,"=""8412614577""","=""""",0,3.46,Newton Compton Editores,...,NaN,2024/05/04,"to-read, thriller","to-read (#113), thriller (#26)",to-read,NaN,NaN,NaN,0,0
2,54304221,How to Not Die Alone: The Surprising Science T...,Logan Ury,"Ury, Logan",NaN,"=""1982120622""","=""9781982120627""",0,4.02,Simon & Schuster,...,NaN,2022/05/08,"to-read, desarrollo","to-read (#5), desarrollo (#7)",to-read,NaN,NaN,NaN,0,0
3,127609209,Corazón en fuera de juego,Nira Strauss,"Strauss, Nira",NaN,"=""8408270591""","=""9788408270591""",0,3.98,Esencia,...,NaN,2024/01/11,"to-read, young-adult","to-read (#28), young-adult (#15)",to-read,NaN,NaN,NaN,0,0
4,49392936,"Bookish and the Beast (Once Upon a Con, #3)",Ashley Poston,"Poston, Ashley",NaN,"=""1683691938""","=""9781683691938""",0,3.78,Quirk Books,...,NaN,2024/01/11,"to-read, young-adult","to-read (#29), young-adult (#14)",to-read,NaN,NaN,NaN,0,0


In [4]:
data.columns

Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Owned Copies'],
      dtype='object')

In [5]:
data.drop (["Book Id","ISBN13","Owned Copies","Read Count","Private Notes", "Spoiler","ISBN","Binding","Publisher","Author l-f"
           ,"Additional Authors","My Review","Bookshelves with positions","Original Publication Year"],axis=1,inplace=True)

In [6]:
data.head()

,Title,Author,My Rating,Average Rating,Number of Pages,Year Published,Date Read,Date Added,Bookshelves,Exclusive Shelf
0,"Heartless Hunter (The Crimson Moth, #1)",Kristen Ciccarelli,0,4.25,416,2024,NaN,2024/05/08,"to-read, fantasy",to-read
1,Una familia casi perfecta,Jane Shemilt,0,3.46,378,2023,NaN,2024/05/04,"to-read, thriller",to-read
2,How to Not Die Alone: The Surprising Science T...,Logan Ury,0,4.02,352,2021,NaN,2022/05/08,"to-read, desarrollo",to-read
3,Corazón en fuera de juego,Nira Strauss,0,3.98,400,2023,NaN,2024/01/11,"to-read, young-adult",to-read
4,"Bookish and the Beast (Once Upon a Con, #3)",Ashley Poston,0,3.78,288,2020,NaN,2024/01/11,"to-read, young-adult",to-read


In [7]:
pendientes=data[data["Exclusive Shelf"]=="to-read"].copy().reset_index()

In [8]:
pendientes.drop (["index","Date Read","My Rating","Exclusive Shelf"],axis=1,inplace=True)

In [9]:
pendientes.head()

,Title,Author,Average Rating,Number of Pages,Year Published,Date Added,Bookshelves
0,"Heartless Hunter (The Crimson Moth, #1)",Kristen Ciccarelli,4.25,416,2024,2024/05/08,"to-read, fantasy"
1,Una familia casi perfecta,Jane Shemilt,3.46,378,2023,2024/05/04,"to-read, thriller"
2,How to Not Die Alone: The Surprising Science T...,Logan Ury,4.02,352,2021,2022/05/08,"to-read, desarrollo"
3,Corazón en fuera de juego,Nira Strauss,3.98,400,2023,2024/01/11,"to-read, young-adult"
4,"Bookish and the Beast (Once Upon a Con, #3)",Ashley Poston,3.78,288,2020,2024/01/11,"to-read, young-adult"


In [10]:
pendientes["Bookshelves"].unique()

array(['to-read, fantasy', 'to-read, thriller', 'to-read, desarrollo',
       'to-read, young-adult', 'to-read, distopia', 'fantasy, to-read',
       'to-read, historica', 'to-read, fantasy, fisico',
       'thriller, to-read', 'to-read, fisico, fantasy'], dtype=object)

Ahora quiero que el genero se separe del to-read y del fisico

In [11]:
pendientes['Bookshelves'] = pendientes['Bookshelves'].str.replace('to-read, ', '').str.replace(', to-read', '').str.replace(', fisico', '').str.replace('fisico, ', '')

In [12]:
pendientes["Bookshelves"].unique()

array(['fantasy', 'thriller', 'desarrollo', 'young-adult', 'distopia',
       'historica'], dtype=object)

¿Autoconclusivo o saga? Quiero añadir otra columna que diga si el libro es autoconclusivo o si es una saga. Los libros que en el titulo tienen un corchete son sagas.

In [13]:
pendientes['Autoconclusivo'] = pendientes['Title'].str.contains('#') 
pendientes['Autoconclusivo'] = pendientes['Autoconclusivo'].map({True: 'No', False: 'Sí'})


In [14]:
pendientes.head()

,Title,Author,Average Rating,Number of Pages,Year Published,Date Added,Bookshelves,Autoconclusivo
0,"Heartless Hunter (The Crimson Moth, #1)",Kristen Ciccarelli,4.25,416,2024,2024/05/08,fantasy,No
1,Una familia casi perfecta,Jane Shemilt,3.46,378,2023,2024/05/04,thriller,Sí
2,How to Not Die Alone: The Surprising Science T...,Logan Ury,4.02,352,2021,2022/05/08,desarrollo,Sí
3,Corazón en fuera de juego,Nira Strauss,3.98,400,2023,2024/01/11,young-adult,Sí
4,"Bookish and the Beast (Once Upon a Con, #3)",Ashley Poston,3.78,288,2020,2024/01/11,young-adult,No


¿Cuál es la media y la mediana de los libros?

In [15]:
pendientes.describe().T

,count,mean,std,min,25%,50%,75%,max
Average Rating,116.0,4.065172,0.287630,3.32,3.8875,4.13,4.28,4.8
Number of Pages,116.0,444.068966,188.026304,0.00,350.0000,404.50,512.00,1243.0
Year Published,116.0,2018.293103,7.162726,1988.00,2016.7500,2021.00,2023.00,2024.0


Cuando tengamos que decidir el número de páginas será mayor o menor de 350. Esto se puede modificar al gusto.

In [16]:
pendientes['longitud'] = pendientes['Number of Pages']>350
pendientes['longitud'] = pendientes['longitud'].map({True: 'Sí', False: 'No'})

In [17]:
pendientes.head()

,Title,Author,Average Rating,Number of Pages,Year Published,Date Added,Bookshelves,Autoconclusivo,longitud
0,"Heartless Hunter (The Crimson Moth, #1)",Kristen Ciccarelli,4.25,416,2024,2024/05/08,fantasy,No,Sí
1,Una familia casi perfecta,Jane Shemilt,3.46,378,2023,2024/05/04,thriller,Sí,Sí
2,How to Not Die Alone: The Surprising Science T...,Logan Ury,4.02,352,2021,2022/05/08,desarrollo,Sí,Sí
3,Corazón en fuera de juego,Nira Strauss,3.98,400,2023,2024/01/11,young-adult,Sí,Sí
4,"Bookish and the Beast (Once Upon a Con, #3)",Ashley Poston,3.78,288,2020,2024/01/11,young-adult,No,No


--------------

Una vez que tenemos el archivo limpio. Pasamos a la clasificación. Hay tres cosas importantes a la hora de elegir libro:
1. ¿Autoconclusio o saga?
2. Número de páginas
3. Género

In [18]:
"""saga = input("¿Quieres que el libro sea una saga? Sí/No: ")
n_paginas = input("¿Quieres que el libro tenga más de 350 páginas? Sí/No: ")
genero = input("¿Qué género quieres? 'fantasy', 'thriller', 'desarrollo', 'young-adult','distopia','historica: ")"""

'saga = input("¿Quieres que el libro sea una saga? Sí/No: ")\nn_paginas = input("¿Quieres que el libro tenga más de 350 páginas? Sí/No: ")\ngenero = input("¿Qué género quieres? \'fantasy\', \'thriller\', \'desarrollo\', \'young-adult\',\'distopia\',\'historica: ")'

In [21]:
from tkinter import *
import pandas as pd

raiz = Tk()
raiz.title("Próxima lectura")

# Crear el marco rosa
marco_rosa = Frame(raiz, width=1200, height=600, bg="pink")
marco_rosa.pack()

# Crear el marco blanco dentro del marco rosa
marco_blanco = Frame(marco_rosa, width=800, height=200, bg="white")
marco_blanco.pack(pady=20, padx=20)

# Crear las etiquetas y los campos de entrada en el marco blanco
cuadroSaga = Entry(marco_blanco)
cuadroSaga.grid(row=0, column=1)

cuadroPags = Entry(marco_blanco)
cuadroPags.grid(row=1, column=1)

cuadroGenero = Entry(marco_blanco)
cuadroGenero.grid(row=2, column=1)

sagaLabel = Label(marco_blanco, text="¿Quieres que el libro sea autoconclusivo? Sí/No:")
sagaLabel.grid(row=0, column=0, sticky="w", padx=10, pady=10)

pagsLabel = Label(marco_blanco, text="¿Quieres que el libro tenga más de 350 páginas? Sí/No:")
pagsLabel.grid(row=1, column=0, sticky="w", padx=10, pady=10)

generoLabel = Label(marco_blanco, text="¿Qué género quieres? (fantasy, thriller, desarrollo, young-adult, distopia, historica): ")
generoLabel.grid(row=2, column=0, sticky="w", padx=10, pady=10)

def codigoBoton():
    # Obtener los valores ingresados por el usuario desde los campos de entrada
    saga = cuadroSaga.get().strip().lower()
    pags = cuadroPags.get().strip().lower()
    genero = cuadroGenero.get().strip().lower()

    # Filtrar el DataFrame según los valores ingresados por el usuario
    libros_filtrados = pendientes
    if saga == "sí":
        libros_filtrados = libros_filtrados[libros_filtrados['Autoconclusivo'].str.lower() == "sí"]
    elif saga == "no":
        libros_filtrados = libros_filtrados[libros_filtrados['Autoconclusivo'].str.lower() == "no"]
    
    if pags == "sí":
        libros_filtrados = libros_filtrados[libros_filtrados['longitud'].str.lower() == "sí"]
    elif pags == "no":
        libros_filtrados = libros_filtrados[libros_filtrados['longitud'].str.lower() == "no"]

    if genero in ["fantasy", "thriller", "desarrollo", "young-adult", "distopia", "historica"]:
        libros_filtrados = libros_filtrados[libros_filtrados['Bookshelves'].str.lower() == genero]
    
    # Depuración: Mostrar DataFrame filtrado en consola
    print("Filtrados:")
    print(libros_filtrados)

    # Seleccionar un libro al azar de los libros filtrados
    if not libros_filtrados.empty:
        libro_aleatorio = libros_filtrados.sample(n=1)
        # Mostrar la información del libro seleccionado en la ventana de la interfaz
        info_libro.set(libro_aleatorio.to_string(index=False))
    else:
        info_libro.set("No se encontraron libros con esos criterios")

# Botón "Siguiente" dentro del marco blanco
botonProcesar = Button(marco_blanco, text="Siguiente", command=codigoBoton)
botonProcesar.grid(row=3, column=0, columnspan=2, pady=10)

# Etiqueta para mostrar la información del libro seleccionado dentro del marco rosa
info_libro = StringVar()
etiqueta_libro = Label(marco_rosa, textvariable=info_libro, bg="pink")
etiqueta_libro.pack(pady=20)

raiz.mainloop()


Filtrados:
                                            Title           Author  \
30      The Pumpkin Spice Café (Dream Harbor, #1)   Laurie Gilmore   
49              The Housemaid (The Housemaid, #1)  Freida McFadden   
103  El secreto de la asistenta (La asistenta #2)  Freida McFadden   

     Average Rating  Number of Pages  Year Published  Date Added Bookshelves  \
30             3.44              267            2023  2024/03/10    thriller   
49             4.33              338            2022  2024/01/31    thriller   
103            4.26              336            2024  2024/04/24    thriller   

    Autoconclusivo longitud  
30              No       No  
49              No       No  
103             No       No  


In [ ]:
filtro = (pendientes['Autoconclusivo'] == saga) & (pendientes['longitud'] == n_paginas) & (pendientes['Bookshelves'] == genero)
libros_filtrados = pendientes[filtro]
libro_aleatorio = libros_filtrados.sample()
libro_aleatorio

In [20]:
libros_filtrados = pendientes[filtro]


NameError: name 'filtro' is not defined

In [ ]:
libro_aleatorio = libros_filtrados.sample()


In [66]:
libro_aleatorio

,Title,Author,Average Rating,Number of Pages,Year Published,Date Added,Bookshelves,Autoconclusivo,longitud
30,"The Pumpkin Spice Café (Dream Harbor, #1)",Laurie Gilmore,3.44,267,2023,2024/03/10,thriller,Sí,No
